# Genetic Algorithm Framework Structure 

1. Genome Representation:

    Each genome represents the weights of the neural network (Linear_QNet).
    We'll store these weights in a flat array (a single vector).

2. Population Initialization:

    Randomly generate initial weights for a population of networks.

3. Fitness Function:

    Run the Snake game using the weights of the network, and use the game score as the fitness.

4. Selection, Crossover, and Mutation:

    Select the best-performing genomes for reproduction.
    Combine and slightly mutate the weights to create a new generation.

5. Evolution Loop:

    Repeat the process for a fixed number of generations.

In [1]:
import torch
import random
import matplotlib.pyplot as plt
import numpy as np
from model import Linear_QNet
from agent import Agent
from snake import SnakeAI

pygame 2.6.0 (SDL 2.28.4, Python 3.9.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Step 1: Implementing the Genetic Algorithm

### 1. Genome Representation and Population Initialization:

In [2]:
# Parameters for the GA
POPULATION_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.1

# Parameters for the Neural Network
INPUT_SIZE = 12
HIDDEN_SIZE = 255
OUTPUT_SIZE = 3

In [3]:
# Flatten the model's parameters into a single array (genome)
def flatten_weights(model):
    # Convert model's parameters to a list of numpy arrays
    return np.concatenate([param.data.cpu().numpy().flatten() for param in model.parameters()])

# Create a new model from a flattened genome
def set_weights(model, genome):
    idx = 0
    for param in model.parameters():
        param_length = np.prod(param.data.shape)
        param.data = torch.tensor(genome[idx:idx + param_length].reshape(param.data.shape))
        idx += param_length

# Initialize a population of random genomes
def initialize_population():
    population = []
    for _ in range(POPULATION_SIZE):
        model = Linear_QNet(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
        genome = flatten_weights(model)
        population.append(genome)
    return population

### 2. Fitness Evaluation

In [4]:
#? We’ll define a function that plays the game with a given genome and returns the fitness:
def evaluate_fitness(genome):
    model = Linear_QNet(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
    set_weights(model, genome)
    agent = Agent()
    agent.model = model
    total_score = 0
    
    for _ in range(5):  # Run multiple games to get a more accurate fitness
        snake = SnakeAI()
        while True:
            state_old = agent.get_state(snake)
            final_move = agent.get_action(state_old)
            reward, done, score = snake.play(final_move)
            if done:
                total_score += score
                break

    return total_score / 5

### 3. Selection Crossover and Mutation

In [5]:
def select_parents(population, fitnesses):
    # Normalize fitnesses to sum to 1 (probabilities)
    fitnesses = fitnesses / np.sum(fitnesses)
    
    # Select two parents based on their fitness probabilities
    selected_indices = np.random.choice(len(population), size=2, p=fitnesses)
    
    # Retrieve the actual genomes corresponding to the selected indices
    parent1 = population[selected_indices[0]]
    parent2 = population[selected_indices[1]]
    
    return parent1, parent2

def crossover(parent1, parent2):
    crossover_point = np.random.randint(0, len(parent1))
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return child

def mutate(genome):
    for i in range(len(genome)):
        if random.random() < MUTATION_RATE:
            genome[i] += np.random.randn() * 0.1  # Small mutation
    return genome


### 4. Evolution Loop

In [6]:
def evolve_population(population):
    best_fitnesses = []
    average_fitnesses = []
    diversity = []

    for generation in range(GENERATIONS):
        fitnesses = np.array([evaluate_fitness(genome) for genome in population])
        new_population = []
        
        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = select_parents(population, fitnesses)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            new_population.append(mutate(child1))
            new_population.append(mutate(child2))
        
        population = new_population
        
        # Record metrics
        best_fitness = np.max(fitnesses)
        avg_fitness = np.mean(fitnesses)
        pop_diversity = np.mean([np.std([genome[i] for genome in population]) for i in range(len(population[0]))])

        best_fitnesses.append(best_fitness)
        average_fitnesses.append(avg_fitness)
        diversity.append(pop_diversity)

        print(f"Generation {generation} | Best Fitness: {best_fitness} | Average Fitness: {avg_fitness} | Diversity: {pop_diversity}")
    
    return population, best_fitnesses, average_fitnesses, diversity
    

if __name__ == "__main__":
    population = initialize_population()
    evolved_population, best_fitnesses, average_fitnesses, diversity = evolve_population(population)
    
    best_genome = max(evolved_population, key=evaluate_fitness)
    
    best_model = Linear_QNet(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
    set_weights(best_model, best_genome)
    best_model.save("best_genetic_model.pth")

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.plot(best_fitnesses)
    plt.title('Best Fitness Over Generations')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')

    plt.subplot(1, 3, 2)
    plt.plot(average_fitnesses)
    plt.title('Average Fitness Over Generations')
    plt.xlabel('Generation')
    plt.ylabel('Average Fitness')

    plt.subplot(1, 3, 3)
    plt.plot(diversity)
    plt.title('Diversity Over Generations')
    plt.xlabel('Generation')
    plt.ylabel('Diversity')

    plt.tight_layout()
    plt.show()